In [1]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
from alphamind.api import *
from PyFin.api import *
from config import *
from models import *
from sqlalchemy import *
from matplotlib import rc
from matplotlib import pyplot as plt

plt.style.use("seaborn-poster")
rc('font', **{'family': 'Microsoft Yahei', 'size': 10})
rc('mathtext', **{'default': 'regular'})
rc('legend', **{'frameon': False})

In [2]:
start_date = '2000-01-01'
end_date = '2019-06-01'
engine = SqlEngine(alpha_db)

universe_name = 'zz500'
freq = '2b'
freq_1 = '3b'
freq_2 = '5b'

/opt/workenv/vision/lib/python3.6/site-packages/psycopg2-2.7.7-py3.6-linux-x86_64.egg/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
universe = Universe(universe_name)
dates = makeSchedule(start_date, end_date, freq, calendar='china.sse')
factor_engine = create_engine(alpha_db)

In [4]:
query = select([Alpha191.trade_date,Alpha191.code,Alpha191.alpha_1, Alpha191.alpha_2, 
                Alpha191.alpha_3, Alpha191.alpha_4, Alpha191.alpha_5, Alpha191.alpha_6,
               Alpha191.alpha_7, Alpha191.alpha_8, Alpha191.alpha_9, Alpha191.alpha_10]).where(Alpha191.trade_date.in_(dates))

factors = pd.read_sql(query, factor_engine)


In [5]:
unicode = lambda code: ('%06d.XSHG' % int(code)) if int(code) < 600000 else ('%06d.XSHE' % int(code))

In [6]:
factors['code'] = factors['code'].apply(unicode)
factors['trade_date'] = factors['trade_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [7]:
zz500_df = pd.read_csv("return_label.csv")
zz500_df = zz500_df.drop(['Unnamed: 0'], axis=1)
len(zz500_df)

573439

In [8]:
tmp = pd.merge(factors, zz500_df, on=['trade_date', 'code'])

In [9]:
tmp = tmp.dropna(axis=0)

In [12]:
tmp = tmp.sort_values(by='trade_date')

In [16]:
tmp.to_csv('traning_sample.csv', index=0)